In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import glob
import random
from sklearn.model_selection import train_test_split
import json
import os
import datetime
import time
import shutil
import pandas as pd

# import wandb

from tqdm import tqdm

In [3]:
BATCH_SIZE = 4

In [4]:
df = pd.read_csv("/media/viktor/T7/gravitational-waves-kaggle-2022/datasets/dataset-v1/data.csv")

In [5]:
# train test split the data and labels
print("[INFO] train test split the data and labels")
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
# del data


[INFO] train test split the data and labels


In [6]:
# Check if data has any nan values
print("[INFO] checking if data has any nan values")
print(np.isnan(X_train).any(), np.isnan(X_test).any())

[INFO] checking if data has any nan values
False False


In [7]:

# Make tf. dataset that repeats and has a batch size of BATCH_SIZE
print("[INFO] Make tf. train dataset that repeats and has a batch size of BATCH_SIZE")
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).repeat().batch(BATCH_SIZE)
del X_train, y_train
print("[INFO] Make tf. test dataset that repeats and has a batch size of BATCH_SIZE")
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).repeat().batch(BATCH_SIZE)
del X_test, y_test

[INFO] Make tf. train dataset that repeats and has a batch size of BATCH_SIZE


2022-12-21 23:07:41.768295: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 23:07:41.827946: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 23:07:41.828250: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 23:07:41.829884: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

: 

: 

In [ ]:
try:
    del X_train, y_train, X_test, y_test
except:
    print("[INFO] failed to delete the arrays!")
    pass

In [ ]:
import gc
gc.collect()

In [ ]:
# load resnet model
model = tf.keras.applications.EfficientNetB0(
    include_top=True,
    weights=None,
    input_shape=(360, 256, 2),
    pooling=None,
    classes=1,
    classifier_activation='sigmoid',
)

In [ ]:
model.predict(tf.ones((1, 360, 256, 2)))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1.E-04),
                # loss is binary crosseentropy, because we have a binary classification problem
                loss= tf.keras.losses.BinaryCrossentropy(),
                metrics=[tf.keras.metrics.AUC(name="auc")]
                )

In [ ]:
model.evaluate(test_dataset.take(16))

In [ ]:
# wandb.login()

In [ ]:

# wandb.init(project="gravitational-waves", entity="viktor-cikojevic")



In [ ]:

history = model.fit(train_dataset, 
        steps_per_epoch=32, 
        epochs=256, 
        validation_data=test_dataset, 
        validation_steps=16,
        # don't shuffle the data
        # shuffle=True,
        callbacks=[
        # WandbCallback(save_model=False), 
           tf.keras.callbacks.ModelCheckpoint(f"best_model.h5", save_best_only=True, monitor='val_auc', mode='max')
            ]
        )

